## Training neural networks
In this notebook we exlpore various Keras functions that are helpful for DNN training. We also note some interesting things about them.

In [3]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [4]:
# overly large model
dnn = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32,32,3])
])

for i in range(20):
    dnn.add(keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))
    
dnn.add(keras.layers.Dense(10, activation='softmax'))

In [5]:
dataset = keras.datasets.cifar10.load_data()
(X_train_full, y_train_full), (X_test, y_test) = dataset

170498071/170498071 [==============================] - 107s 1us/step


In [6]:
X_train = X_train_full[:40000]
y_train = y_train_full[:40000]
X_val = X_train_full[40000:]
y_val = y_train_full[40000:]
X_val.shape

(10000, 32, 32, 3)

### Early stopping
- Probably outdated.

In [111]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

optimizer = keras.optimizers.Nadam(lr=5e-4)
dnn.compile(optimizer=optimizer, 
            loss=keras.losses.sparse_categorical_crossentropy, 
            metrics=['accuracy'], 
            )
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
dnn.fit(X_train, y_train,  epochs=100,
        validation_data=(X_val, y_val),
        callbacks = [early_stopping_cb]
       )
dnn.evaluate(X_val, y_val)

Epoch 1/100
1250/1250 [==============================] - 15s 12ms/step - loss: 2.9605 - accuracy: 0.2097 - val_loss: 1.9670 - val_accuracy: 0.2583
Epoch 2/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.9348 - accuracy: 0.2842 - val_loss: 1.8829 - val_accuracy: 0.3001
Epoch 3/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.8653 - accuracy: 0.3114 - val_loss: 1.8350 - val_accuracy: 0.3299
Epoch 4/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.8247 - accuracy: 0.3322 - val_loss: 1.8243 - val_accuracy: 0.3346
Epoch 5/100
1250/1250 [==============================] - 16s 13ms/step - loss: 1.7916 - accuracy: 0.3465 - val_loss: 1.7851 - val_accuracy: 0.3523
Epoch 6/100
1250/1250 [==============================] - 16s 13ms/step - loss: 1.7651 - accuracy: 0.3571 - val_loss: 1.7951 - val_accuracy: 0.3517
Epoch 7/100
1250/1250 [==============================] - 16s 13ms/step - loss: 1.7344 - accuracy: 0.3678 - val_loss: 1

[1.5680395364761353, 0.43470001220703125]

### Batch Normalization

- Note: this is essentially scaling the data *on all layers* which enourmously helps gradient descent. Therefore: BatchNorm layers are included in almost every architecture now (ex. Transformers)

In [81]:
dnn2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32,32,3])
])

for i in range(20):
    dnn2.add(keras.layers.BatchNormalization())
    dnn2.add(keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))
    
dnn2.add(keras.layers.Dense(10, activation='softmax'))

In [82]:
dnn2.compile(optimizer="nadam", 
            loss=keras.losses.sparse_categorical_crossentropy, 
            metrics=['accuracy'], 
            )
dnn2.fit(X_train, y_train,  epochs=30,
        validation_data=(X_val, y_val),
        callbacks = [keras.callbacks.EarlyStopping(patience=10)]
       )

Epoch 1/30
1250/1250 [==============================] - 21s 17ms/step - loss: 1.9431 - accuracy: 0.3015 - val_loss: 1.7523 - val_accuracy: 0.3659
Epoch 2/30
1250/1250 [==============================] - 21s 17ms/step - loss: 1.7417 - accuracy: 0.3784 - val_loss: 1.6397 - val_accuracy: 0.4149
Epoch 3/30
1250/1250 [==============================] - 22s 18ms/step - loss: 1.6646 - accuracy: 0.4096 - val_loss: 1.5739 - val_accuracy: 0.4448
Epoch 4/30
1250/1250 [==============================] - 22s 18ms/step - loss: 1.6101 - accuracy: 0.4283 - val_loss: 1.5421 - val_accuracy: 0.4526
Epoch 5/30
1250/1250 [==============================] - 22s 18ms/step - loss: 1.5612 - accuracy: 0.4505 - val_loss: 1.5358 - val_accuracy: 0.4576
Epoch 6/30
1250/1250 [==============================] - 23s 18ms/step - loss: 1.5295 - accuracy: 0.4576 - val_loss: 1.4999 - val_accuracy: 0.4726
Epoch 7/30
1250/1250 [==============================] - 23s 19ms/step - loss: 1.4956 - accuracy: 0.4713 - val_loss: 1.5275 -

In [6]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_val_scaled = (X_val - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

### Different nonlinearities 

In [7]:
# SELU
dnn3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32,32,3])
])

for i in range(20):
    dnn3.add(keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'))
    
dnn3.add(keras.layers.Dense(10, activation='softmax'))

optimizer = keras.optimizers.Nadam(lr=8e-4)
dnn3.compile(optimizer=optimizer, 
            loss=keras.losses.sparse_categorical_crossentropy, 
            metrics=['accuracy'], 
            )
cb = keras.callbacks.EarlyStopping(patience=10)



dnn3.fit(X_train_scaled, y_train,  epochs=100,
        validation_data=(X_val_scaled, y_val),
        callbacks = [cb]
       )

Epoch 1/100
1250/1250 [==============================] - 17s 13ms/step - loss: 1.9376 - accuracy: 0.3032 - val_loss: 1.7903 - val_accuracy: 0.3711
Epoch 2/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.7254 - accuracy: 0.3894 - val_loss: 1.6934 - val_accuracy: 0.3971
Epoch 3/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.6292 - accuracy: 0.4233 - val_loss: 1.6822 - val_accuracy: 0.4236
Epoch 4/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.5785 - accuracy: 0.4462 - val_loss: 1.6038 - val_accuracy: 0.4362
Epoch 5/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.5211 - accuracy: 0.4661 - val_loss: 1.5563 - val_accuracy: 0.4500
Epoch 6/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.4748 - accuracy: 0.4851 - val_loss: 1.5592 - val_accuracy: 0.4550
Epoch 7/100
1250/1250 [==============================] - 15s 12ms/step - loss: 1.4358 - accuracy: 0.4969 - val_loss: 1

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

optimizer = keras.optimizers.Nadam(lr=1e-2, decay=)
dnn.compile(optimizer=optimizer, 
            loss=keras.losses.sparse_categorical_crossentropy, 
            metrics=['accuracy'], 
            )
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
dnn.fit(X_train, y_train,  epochs=100,
        validation_data=(X_val, y_val),
        callbacks = [early_stopping_cb]
       )
dnn.evaluate(X_val, y_val)